In [17]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

In [18]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes=100)
y_test = keras.utils.to_categorical(y_test, num_classes=100)

In [19]:
batch_size = 128
epochs = 180
n_classes = 100
learning_rate = 0.1

In [20]:
class Resnet:
    def __init__(self, size=44, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=100):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

        network = Activation('relu')(network)
        network = AveragePooling2D(pool_size=network.shape[1])(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              strides=1, activation='relu', normalize_batch=True):
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer="he_normal",
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    
def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 100:
        pass
    elif epoch > 100 and epoch <= 150:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

In [21]:
resnet = Resnet()

model = resnet.get_model()

optimizer = SGD(learning_rate=learning_rate, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_90 (Conv2D)             (None, 32, 32, 16)   448         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_48 (BatchN  (None, 32, 32, 16)  64          ['conv2d_90[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_46 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_48[0]

In [22]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)

In [23]:
wandb.init(project="CIFAR100", entity="hcim", name='ResNet-ReLU')

accuracy,▇▂▇▆▄▆▃▇▅▃▇▅▄▇▅▅▃▆▅█▄▄▆▃▄▄▃▅▅▅▃▅▄▃▇▄▆▁▄▆
loss,█▇▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.00958
loss,4.60832
val_accuracy,0.01
val_loss,4.60675


In [24]:
lr_scheduler = LearningRateScheduler(learning_rate_schedule)
callbacks = [WandbCallback(), lr_scheduler]

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=4,
                             height_shift_range=4,
                             horizontal_flip=True,
                             preprocessing_function=get_random_eraser(p=1, pixel_level=True))
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_test, y_test),
          epochs=epochs, workers=4,
          callbacks=callbacks)

Learning rate: 0.1
Epoch 1/180
391/391 [==============================] - 22s 50ms/step - loss: 4.9476 - accuracy: 0.0092 - val_loss: 4.9008 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 2/180
391/391 [==============================] - 19s 49ms/step - loss: 4.8808 - accuracy: 0.0096 - val_loss: 4.8583 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 3/180
391/391 [==============================] - 20s 50ms/step - loss: 4.8415 - accuracy: 0.0091 - val_loss: 4.8217 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 4/180
391/391 [==============================] - 20s 50ms/step - loss: 4.8076 - accuracy: 0.0085 - val_loss: 4.7905 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 5/180
391/391 [==============================] - 20s 50ms/step - loss: 4.7787 - accuracy: 0.0104 - val_loss: 4.7637 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 6/180
391/391 [==============================] - 20s 50ms/step - loss: 4.7539 - accuracy: 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



391/391 [==============================] - 20s 51ms/step - loss: 4.6053 - accuracy: 0.0092 - val_loss: 4.6052 - val_accuracy: 0.0100 - lr: 0.0010
Learning rate: 0.001
Epoch 162/180
391/391 [==============================] - 20s 51ms/step - loss: 4.6053 - accuracy: 0.0093 - val_loss: 4.6052 - val_accuracy: 0.0100 - lr: 0.0010
Learning rate: 0.001
Epoch 163/180
391/391 [==============================] - 20s 50ms/step - loss: 4.6053 - accuracy: 0.0096 - val_loss: 4.6052 - val_accuracy: 0.0100 - lr: 0.0010
Learning rate: 0.001
Epoch 164/180
391/391 [==============================] - 20s 50ms/step - loss: 4.6053 - accuracy: 0.0098 - val_loss: 4.6052 - val_accuracy: 0.0100 - lr: 0.0010
Learning rate: 0.001
Epoch 165/180
391/391 [==============================] - 20s 51ms/step - loss: 4.6053 - accuracy: 0.0089 - val_loss: 4.6052 - val_accuracy: 0.0100 - lr: 0.0010
Learning rate: 0.001
Epoch 166/180
391/391 [==============================] - 20s 50ms/step - loss: 4.6053 - accuracy: 0.0089 - va